In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import package

In [2]:
# !pip install pandas facenet_pytorch torch==2.2.0 torchvision==0.17.0 numpy==1.24.0 matplotlib pillow==10.2.0 pytest-shutil opencv-python

In [3]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os
import shutil

from torchvision import datasets, transforms
from copy import deepcopy
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np
import os

from sklearn.metrics import classification_report
import seaborn as sns

# Check CUDA availability

In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

## a/ Get dataframe

In [6]:
df = pd.read_csv("/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/data/lfw_allnames.csv")
df

,name,images
0,AJ_Cook,1
1,AJ_Lamas,1
2,Aaron_Eckhart,1
3,Aaron_Guiel,1
4,Aaron_Patterson,1
...,...,...
5744,Zorica_Radovic,1
5745,Zulfiqar_Ahmed,1
5746,Zumrati_Juma,1
5747,Zurab_Tsereteli,1


## b/ Read .jpg files

In [7]:
# img = mpimg.imread('../content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/data/lfw-deepfunneled/Zinedine_Zidane/Zinedine_Zidane_0004.jpg')
# plt.imshow(img)

In [8]:
# # Load an image
# image = Image.open('../data/lfw-deepfunneled/Zinedine_Zidane/Zinedine_Zidane_0004.jpg')

# # Get the size of the image
# width, height = image.size

# print(f"{width}x{height}")

# Select data for modelisation

### a/ Select randomly 100 individuals that contain between 5 and 15 images

In [9]:
# Select randomly 100 individuals that contain between 5 and 15 images :

df = df.loc[(df["images"] >= 5) & (df["images"] <= 15), ["name", "images"]][50:150]
df

,name,images
847,Cesar_Gaviria,8
856,Chanda_Rubin,5
876,Charles_Moose,13
883,Charles_Taylor,9
897,Charlton_Heston,6
...,...,...
2630,Joan_Laporta,9
2659,Joe_Lieberman,13
2679,John_Abizaid,9
2680,John_Allen_Muhammad,11


In [10]:
list_name = list(df["name"])
list_name

['Cesar_Gaviria',
 'Chanda_Rubin',
 'Charles_Moose',
 'Charles_Taylor',
 'Charlton_Heston',
 'Chen_Shui-bian',
 'Choi_Sung-hong',
 'Christine_Baumgartner',
 'Christine_Todd_Whitman',
 'Ciro_Gomes',
 'Clara_Harris',
 'Claudia_Pechstein',
 'Clay_Aiken',
 'Clint_Eastwood',
 'Colin_Farrell',
 'Colin_Montgomerie',
 'Condoleezza_Rice',
 'Costas_Simitis',
 'Cruz_Bustamante',
 'David_Anderson',
 'David_Heymann',
 'David_Nalbandian',
 'David_Trimble',
 'David_Wells',
 'Dennis_Hastert',
 'Dennis_Kucinich',
 'Denzel_Washington',
 'Diana_Krall',
 'Dick_Cheney',
 'Dominique_de_Villepin',
 'Edmund_Stoiber',
 'Eduard_Shevardnadze',
 'Eduardo_Duhalde',
 'Edward_Lu',
 'Elizabeth_Hurley',
 'Elizabeth_Smart',
 'Elsa_Zylberstein',
 'Elton_John',
 'Emanuel_Ginobili',
 'Emma_Watson',
 'Enrique_Bolanos',
 'Erika_Harold',
 'Fernando_Gonzalez',
 'Fernando_Henrique_Cardoso',
 'Frank_Solich',
 'Fujio_Cho',
 'Gene_Robinson',
 'Geoff_Hoon',
 'George_Clooney',
 'George_HW_Bush',
 'George_Lopez',
 'George_Pataki',
 

### b/ Create train folder

In [11]:
# # create train folder (for modelisation) :

# for name in list_name :
#     shutil.copytree(f"../data/lfw-deepfunneled/{name}", f"../data/lfw_01_train/{name}", dirs_exist_ok=True)

### c/ Create test folder

In [12]:
# data_train_path = f"../data/lfw_01_train/"
# data_test_path = f"../data/lfw_03_test/"

# for name in list_name :

#     train_name_path = data_train_path + name + "/"
#     test_name_path = data_test_path + name + "/"

#     # make directory for the name in data/lfw_03_test/ folder :
#     os.makedirs(os.path.dirname(test_name_path), exist_ok=True)

#     # get all file of the indivual folder :
#     list_file = os.listdir(train_name_path)

#     # get num of .jpg file :
#     len_picture = len(list_file)

#     # get the number of picture to extract for data_test :
#     len_test = round(len_picture*0.4)

#     # list of picture to move to data_test :
#     list_file_test = list_file[:len_test]

#     # move train picture to data_test :
#     for picture_file in list_file_test :
#         shutil.move(train_name_path + picture_file, test_name_path + picture_file)


### d/ Create Val folder

In [13]:
# data_test_path = f"../data/lfw_03_test/"
# data_val_path = f"../data/lfw_02_val/"


# for name in list_name :

#     test_name_path = data_test_path + name + "/"
#     val_name_path = data_val_path + name + "/"

#     # make directory for the name in data/lfw_02_val/ folder :
#     os.makedirs(os.path.dirname(val_name_path), exist_ok=True)

#     # get a list of all file of the indivual folder :
#     list_file = os.listdir(test_name_path)

#     # get num of .jpg file :
#     len_picture = len(list_file)

#     # get the number of picture to extract for data_test :
#     len_val = round(len_picture*0.5)

#     # list of picture to move to data_test :
#     if len_val == 0 :
#         list_file_val = list_file[:1]
#     else :
#         list_file_val = list_file[:len_val]

#     # move train picture to data_test :
#     for picture_file in list_file_val :
#         shutil.move(test_name_path + picture_file, val_name_path + picture_file)


# Pre processing

### a/ Détection et alignement avec MTCNN

In [14]:
# Détection et alignement avec MTCNN
mtcnn = MTCNN(image_size=160, margin=20, keep_all=False, device=device)

# Dataset personnalisé
class FaceDataset(Dataset):
    def __init__(self, image_folder, transform=None, fallback_transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.fallback_transform = fallback_transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}
        # self.idx_to_class = {}

        for idx, class_name in enumerate(os.listdir(image_folder)):
            class_path = os.path.join(image_folder, class_name)
            if os.path.isdir(class_path):
                self.class_to_idx[class_name] = idx
                for img_file in os.listdir(class_path):
                    if img_file.endswith(('.jpg', '.png', '.jpeg')):
                        self.image_paths.append(os.path.join(class_path, img_file))
                        self.labels.append(idx)

        self.num_classes = len(self.class_to_idx)
        self.idx_to_class = {idx: name for name, idx in self.class_to_idx.items()}
        self.class_names = [self.idx_to_class[i] for i in range(self.num_classes)]


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert('RGB')

        # Détection du visage :
        face = mtcnn(img)

        if face is None:
          if self.fallback_transform:
            face = self.fallback_transform(img)
          else :
            return self.__getitem__((idx + 1) % len(self))

        if self.transform:
            face = self.transform(face)

        return face, label


### b/ Pré-traitement et augmentation

In [15]:
mean = [0.6071, 0.4609, 0.3944]
std = [0.2457, 0.2175, 0.2129]

fallback_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
    ])

transform_train = transforms.Compose([
    # artificial data increasing :
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2),
    #transforms.ToTensor(),
    #transforms.resize(160,160),
    transforms.Normalize(mean=mean, std=std)
])

transform_val = transforms.Compose([
    #transforms.ToTensor(),
    #transforms.resize(160,160),
    transforms.Normalize(mean=mean, std=std)
])

transform_test = transforms.Compose([
    #transforms.ToTensor(),
    #transforms.resize(160,160),
    transforms.Normalize(mean=mean, std=std)
])

### c/ Creation des datasets et dataloader

In [16]:
train_dataset = FaceDataset(image_folder='/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/data/lfw_01_train', transform=transform_train) # , fallback_transform=fallback_transform
val_dataset = FaceDataset(image_folder='/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/data/lfw_02_val', transform=transform_val) # , fallback_transform=fallback_transform
test_dataset = FaceDataset(image_folder='/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/data/lfw_03_test', transform=transform_test) # , fallback_transform=fallback_transform

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


# Fine-tuning

### a/ import model

In [17]:
# Créer le modèle en ajoutant une tete de classif :
class FaceClassifier(nn.Module):
    def __init__(self, base_model, num_classes):
        super(FaceClassifier, self).__init__()
        self.base = base_model # deepcopy(base_model)

        # Ajout tete de classification :
        self.classifier = nn.Linear(512, num_classes)

        # # modify classif head :
        # self.base.logits = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.base(x)
        return self.classifier(x)
        # return self.base.logits(x)


# instanciate the model :
num_classes = train_dataset.num_classes
base_model = InceptionResnetV1(pretrained='vggface2', classify=False)
# base_model.classifier = nn.Linear(512, num_classes)
model = FaceClassifier(base_model, num_classes)

### b/ train

In [18]:
def train_model(model, train_loader, val_loader, num_epochs=20, patience=5, save_path="/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/model/best_fc_param.pt"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    best_f1 = 0
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        # train
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch {epoch+1} - F1-score: {f1:.4f}")

        # Early stopping
        if f1 > best_f1:
            best_f1 = f1
            epochs_no_improve = 0
            torch.save(model.state_dict(), save_path)
            print("Nouveau meilleur modèle sauvegardé !")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping")
                break

    print("Entraînement terminé. Meilleur F1-score :", best_f1)


In [21]:
train_model(model, train_loader, val_loader, num_epochs=10, patience=5)

Epoch 1 - F1-score: 0.1827
Nouveau meilleur modèle sauvegardé !
Epoch 2 - F1-score: 0.2177
Nouveau meilleur modèle sauvegardé !
Epoch 3 - F1-score: 0.2057
Epoch 4 - F1-score: 0.2077
Epoch 5 - F1-score: 0.2057
Epoch 6 - F1-score: 0.2057
Epoch 7 - F1-score: 0.2063
Early stopping
Entraînement terminé. Meilleur F1-score : 0.21766666666666665


In [20]:
eee

NameError: name 'eee' is not defined

# Evaluate model on test set (tester si c'est le f1 = f1_score(all_labels, all_preds, average="macro") # 'macro' qui deconne)

In [ ]:
def evaluate_model(model, test_loader, checkpoint_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.load_state_dict(torch.load(checkpoint_path))
    model = model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # f1 = f1_score(all_labels, all_preds, average="macro") # 'macro'
    #print(f"F1-score (test set): {f1:.4f}")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # return f1


In [ ]:
evaluate_model(model, test_loader, "/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/model/best_fc_param.pt", train_dataset.class_names)

In [ ]:
eeeeeeeeeeee

# Save model (ones the evaluation score is good)

In [ ]:
# Sauvegarde complète du modèle (architecture + poids)
torch.save(model, "/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/model/best_fc_model.pt")

# Test model on one example picture

In [ ]:
# load image :
img = Image.open('/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/data/lfw-deepfunneled/Fujio_Cho/Fujio_Cho_0004.jpg').convert('RGB')
img

In [ ]:
# load model :
best_model = torch.load("./content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/model/best_fc_model.pt")
best_model.eval()

In [ ]:
train_dataset.idx_to_class

In [ ]:
# pre processing parameters :
mean = [0.6071, 0.4609, 0.3944]
std = [0.2457, 0.2175, 0.2129]

fallback_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
    ])

transform = transforms.Compose([
    # transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load Face detection model :
mtcnn = MTCNN(image_size=160, device=device)

# dict_class_name :
dict_class_names = train_dataset.idx_to_class



# predict :
def predict(image_path, model, dict_class_names):
    image = Image.open(image_path).convert('RGB')
    face = mtcnn(image)
    if face is None:
        face = self.fallback_transform(image)
        print("Aucun visage détecté par MTCNN")

    face = transform(face).unsqueeze(0)  # [1, 3, 160, 160]
    face = face.to(next(model.parameters()).device)

    with torch.no_grad():
        outputs = model(face)
        predicted = torch.argmax(outputs, dim=1).item()

    print(f"Prédiction : {dict_class_names[predicted]}")
    return predicted

In [ ]:
predict('/content/drive/MyDrive/Data_science/PROJET_perso/facial_recognition_project-main-Backend-data/Backend/data/lfw-deepfunneled/Fujio_Cho/Fujio_Cho_0004.jpg', best_model, dict_class_names)